In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from lightgbm.engine import train
import pandas as pd
from sklearn.preprocessing import label_binarize
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np

SEED = 0

df = pd.read_csv("../input/credit-card-customers/BankChurners.csv", index_col=["CLIENTNUM"])

df = df.iloc[:, :-2]

y = df["Attrition_Flag"]
X = df.drop("Attrition_Flag", axis=1)

y = label_binarize(y, classes=["Existing Customer", "Attrited Customer"]).reshape(-1)

X[
    ["Gender", "Education_Level", "Marital_Status", "Income_Category", "Card_Category"]
] = X[
    ["Gender", "Education_Level", "Marital_Status", "Income_Category", "Card_Category"]
].astype(
    "category"
)

X["Education_Level"] = X["Education_Level"].cat.reorder_categories(
    [
        "Unknown",
        "Uneducated",
        "High School",
        "College",
        "Graduate",
        "Post-Graduate",
        "Doctorate",
    ],
    ordered=True,
)

X["Marital_Status"] = X["Marital_Status"].cat.reorder_categories(
    ["Unknown", "Single", "Divorced", "Married"], ordered=True
)

X["Income_Category"] = X["Income_Category"].cat.reorder_categories(
    [
        "Unknown",
        "Less than $40K",
        "$40K - $60K",
        "$60K - $80K",
        "$80K - $120K",
        "$120K +",
    ],
    ordered=True,
)

X["Card_Category"] = X["Card_Category"].cat.reorder_categories(
    ["Blue", "Silver", "Gold", "Platinum",], ordered=True
)


X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=SEED)
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

params = {
    "objective": "binary",
    "unbalance": True,
    "eta": 2e-2,
}
model = lgb.train(
    params, dtrain, num_boost_round=10000, valid_sets=dvalid, early_stopping_rounds=50,
)

y_pred = model.predict(X_valid)
print(f"AUC: {roc_auc_score(y_valid, y_pred):.4f}")
